# Classifier

## Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import sys
from tqdm.notebook import tqdm, trange
from time import sleep
from torchvision.io import read_image
from torchvision.transforms import ToTensor
from torchvision import transforms
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from collections import Counter
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from sklearn.model_selection import StratifiedKFold

import winsound


In [2]:
seed = 23
save_model = True
load_model = False

models_directory = "final_models"
metrics_directory = "final_metrics"
metrics_name = "metrics_filtered_early.csv"
dataset_name = "Fer2013_filtered_uniform_aug_surprise100_identity05"
train_df_path = f"../datasets/{dataset_name}/train"
test_df_path = f"../datasets/{dataset_name}/test"

batch_size = 64

In [3]:
today = datetime.datetime.now()
date_time = today.strftime("%m-%d-%Y_%H-%M-%S")
print("date and time:",date_time)

date and time: 09-27-2022_09-19-15


In [4]:
if not os.path.isdir(models_directory):
    os.makedirs(name = models_directory)
models_directory_data = os.path.join(models_directory,dataset_name+"_"+date_time)
if not os.path.isdir(models_directory_data):
    os.makedirs(name = models_directory_data)

## Functions definition

In [5]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [6]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit(epochs, patience, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD, scheduler_func=None, fold = None):
    history = []
    optimizer = opt_func(model.parameters(), lr, betas=(0.5,0.999))
    scheduler = scheduler_func(optimizer, mode='min', patience=3, factor = 0.5) # Reduce lr by half
    min_val_loss = sys.float_info.max
    patience_cnt = 0
    for epoch in tqdm(range(epochs), desc = "Current Epoch"):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in tqdm(train_loader, desc = f"Epoch: {epoch}", leave= False):
            optimizer.zero_grad()
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()

        scheduler.step(result['val_loss'])
        result['lrate'] = get_lr(optimizer)

        if epoch > 0 and min_val_loss - result['val_loss'] >= 1e-4:
            patience_cnt = 0
        else:
            patience_cnt += 1
            
        if result['val_loss'] <= min_val_loss:
            min_val_loss = result['val_loss']
            min_val_epoch = epoch
            model_path = os.path.join(models_directory_data,f"classifier_fold_{fold}.pt")
            torch.save(model.state_dict(), model_path)

        if patience_cnt >= patience:
            print('Early stopping')
            break

        model.epoch_end(epoch, result)
        history.append(result)
    model.load_state_dict(torch.load(os.path.join(models_directory_data,f"classifier_fold_{fold}.pt")))
    return history,min_val_epoch

## Class definition

In [7]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [8]:
class Metric():
    def __init__(self, name):
        self.name = name
    
    def eval(self, outputs, labels):
        pass

In [9]:
class PRECuracy(Metric):
    
    def eval(self, outputs, labels):
        _, preds = torch.max(outputs, dim=1)
        return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [10]:
class ImageClassificationBase(nn.Module):
    
    def __init__(self, loss_function, metrics):
        super().__init__()
        self.loss_function = loss_function
        self.metrics = metrics
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = self.loss_function(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = self.loss_function(out, labels)   # Calculate loss
        result = {'val_loss': loss.detach()}
        
        for m in self.metrics:
            result[m.name] = m.eval(out, labels)           # Calculate metrics
            
        return result
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        
        result = {'val_loss': epoch_loss.item()}
        
        for m in self.metrics:
            batch = [x[m.name] for x in outputs]
            epoch = torch.stack(batch).mean()      # Combine metrics
            result[m.name] = epoch.item()
            
        return result
    
    def epoch_end(self, epoch, result):
        out = f"Epoch [{epoch}]"
        vals = list(result.keys())
        for v in vals:
            out += f", {v}: {result[v]:.3e}"
        print(out)
        

In [11]:
class Net(ImageClassificationBase):
    
    def __init__(self, loss_function, metrics, out_size):
        super().__init__(loss_function, metrics)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2, padding=1)
        self.norm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=2, padding=1)
        self.norm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.norm3 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(in_features=128*5*5, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=256)
        self.fc = nn.Linear(256, out_size)

    def forward(self, input):
        output = self.norm1(self.pool1(F.relu(self.conv1(input))))
        output = self.norm2(self.pool2(F.relu(self.conv2((output)))))
        output = self.norm3(self.pool3(F.relu(self.conv3((output)))))
        #print(output.shape)
        output = output.view(-1, 128*5*5)
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))
        output = self.fc(output)
        output = F.softmax(input = output, dim=-1)
        return output

In [12]:
device = get_default_device()
print(device)

cuda


## Dataset loading

In [13]:
df_train = ImageFolder(root=train_df_path, transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                     transforms.ToTensor()]))
df_test = ImageFolder(root=test_df_path, transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                     transforms.ToTensor()]))

In [14]:
train_dl = DeviceDataLoader(DataLoader(df_train, batch_size=batch_size),device)
test_dl = DeviceDataLoader(DataLoader(df_test, batch_size=batch_size, shuffle=False),device)

In [15]:
classes = df_train.classes
classes

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [16]:
counts = dict(Counter(df_train.targets))
print(counts)
weights = np.array(list(counts.values()))
weights = torch.Tensor( min(weights)/weights)
print(weights)

{0: 3715, 1: 321, 2: 3320, 3: 7867, 4: 4963, 5: 4679, 6: 3476}
tensor([0.0864, 1.0000, 0.0967, 0.0408, 0.0647, 0.0686, 0.0923])


## Model design and Training

In [17]:
loss_function = nn.CrossEntropyLoss(weight = weights)
metrics = []
optimizer = optim.Adam
scheduler = optim.lr_scheduler.ReduceLROnPlateau

lr = 0.0001
num_epochs = 100
patience = 5

In [18]:
splits = 10
skf = StratifiedKFold(n_splits=splits, random_state=seed, shuffle=True)
histories = []

for fold,(train_idx,val_idx) in enumerate(skf.split(df_train,train_dl.dl.dataset.targets)):
    print('------------fold no---------{}----------------------'.format(fold))
    
    net = Net(loss_function, metrics, len(classes))
    net.to(device)
    #summary(net,(1,48,48))
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_fold_dl = DeviceDataLoader(DataLoader(df_train, batch_size=batch_size,  sampler=train_subsampler),device)
    val_fold_dl = DeviceDataLoader(DataLoader(df_train, batch_size=batch_size,  sampler=val_subsampler),device)
    #print(np.unique(np.array(train_fold_dl.dl.dataset.targets)[train_subsampler.indices],return_counts=True))
    #print(np.unique(np.array(train_fold_dl.dl.dataset.targets)[test_subsampler.indices],return_counts=True))
    #print(train_dl[train_subsampler.indices])
    history = dict()
    history["losses"], history["epoch"] = fit(num_epochs, patience, lr, net, train_fold_dl, val_fold_dl, optimizer, scheduler, fold)
    history["model"] = net
    histories.append(history)

summary(net,(1,48,48))

------------fold no---------0----------------------


Current Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0:   0%|          | 0/399 [00:00<?, ?it/s]

Epoch [0], val_loss: 1.645e+00, train_loss: 1.765e+00, lrate: 1.000e-04


Epoch: 1:   0%|          | 0/399 [00:00<?, ?it/s]

...

Epoch [23], val_loss: 1.511e+00, train_loss: 1.247e+00, lrate: 5.000e-05


Epoch: 24:   0%|          | 0/399 [00:00<?, ?it/s]

Epoch [24], val_loss: 1.512e+00, train_loss: 1.245e+00, lrate: 2.500e-05


Epoch: 25:   0%|          | 0/399 [00:00<?, ?it/s]

Early stopping
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 46, 46]             320
         MaxPool2d-2           [-1, 32, 23, 23]               0
       BatchNorm2d-3           [-1, 32, 23, 23]              64
            Conv2d-4           [-1, 64, 21, 21]          18,496
         MaxPool2d-5           [-1, 64, 11, 11]               0
       BatchNorm2d-6           [-1, 64, 11, 11]             128
            Conv2d-7            [-1, 128, 9, 9]          73,856
         MaxPool2d-8            [-1, 128, 5, 5]               0
       BatchNorm2d-9            [-1, 128, 5, 5]             256
           Linear-10                  [-1, 256]         819,456
           Linear-11                  [-1, 256]          65,792
           Linear-12                    [-1, 7]           1,799
Total params: 980,167
Trainable params: 980,167
Non-trainable params: 0
----------------

## Save/Load model

In [19]:
if save_model:
    for fold,h in enumerate(histories):
        model_path = os.path.join(models_directory_data,f"classifier_fold_{fold}.pt")
        torch.save(h["model"].state_dict(), model_path)

In [20]:
if load_model:
    histories = []
    for fold, model in enumerate(os.listdir(models_directory_data)):
        net = Net(loss_function, metrics, len(classes))
        net.load_state_dict(torch.load(os.path.join(models_directory_data,model)))
        net.eval()
        net.cuda()
        histories.append( {"model": net } )

## Model testing

In [21]:
def get_model_predicitons(model, dataset):
    y_test = []
    y_scores = []

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in tqdm(dataset):
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = model(images)

            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)

            y_test.extend(labels.tolist())
            y_scores.extend(outputs.tolist())

    y_test = np.array(y_test)
    y_scores = np.array(y_scores)

    return y_test, y_scores


### Network PRECuracy

In [22]:
def evaluate_accuracy(y_test, y_scores, classes, verbose=False):
    # Overall accuracy

    n_test = y_test.shape[0]
    predicted = np.argmax(y_scores,axis=1)
    mean_acc = np.sum(predicted == y_test) / n_test
    class_acc = []
    
    # Accuracy per class

    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # collect the correct predictions for each class
    for label, prediction in zip(y_test, predicted):
        if label == prediction:
            correct_pred[classes[label]] += 1
        total_pred[classes[label]] += 1

    for classname, correct_count in correct_pred.items():
        accuracy = float(correct_count) / total_pred[classname]
        class_acc.append(accuracy)

    if verbose:
        print(f'Overall accuracy: {100 * mean_acc } %')

        print("Accuracy per class:")
        for i,c in enumerate(classes):
            print(f'{c:9s} : {class_acc[i] * 100:.1f} %')
        
    return mean_acc, class_acc

# ROC/AUC

In [23]:

def evaluate_roc_auc(y_test, y_scores, classes, plot=False, verbose=False):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    ths = dict()
    roc_auc = dict()
    for i,_ in enumerate(classes):
        fpr[i], tpr[i], ths[i] = roc_curve(y_test == i, y_scores[:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    if plot:
        # Plot of a ROC curve for a specific class
        for i,c in enumerate(classes):
            plt.figure()
            plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC {c}')
            plt.legend(loc="lower right")
            plt.show()

    mean_auc = np.mean(list(roc_auc.values()))

    if verbose:
        print(f"Mean AUC: { mean_auc * 100} %")

        print("Per classs AUC:")
        for i,c in enumerate(classes):
            print(f'{c:9s} : {roc_auc[i]*100:.2f} %')

    return mean_auc, roc_auc

In [24]:
y_test = dict()
y_scores = dict()

for fold,h in enumerate(histories):
    y_test[fold], y_scores[fold] = get_model_predicitons(h["model"],test_dl)

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [25]:
columns=["mean AUC", "angry AUC", "disgust AUC", "fear AUC", "happy AUC", "neautral AUC", "sad AUC", "surprise AUC", "mean PREC", "angry PREC", "disgust PREC", "fear PREC", "happy PREC", "neautral PREC", "sad PREC", "surprise PREC"]
df = pd.DataFrame(columns = columns)

for fold,_ in enumerate(histories):
    mean_acc,class_acc = evaluate_accuracy(y_test[fold], y_scores[fold], classes)
    mean_auc,roc_auc = evaluate_roc_auc(y_test[fold], y_scores[fold], classes)

    exp = [mean_auc]
    exp += list(roc_auc.values())
    exp += [mean_acc]
    exp += class_acc

    df_new = pd.DataFrame( np.array([exp]),[f"{dataset_name} fold {fold}"], columns=columns)
    df = pd.concat([df,df_new])
    df = df.astype('float')

mean_df = df.mean(0).to_frame().transpose().set_index(pd.Index([f"{dataset_name} mean"]))
df = pd.concat([df,mean_df])
df


,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Fer2013_filtered_uniform_aug_surprise100_identity05 fold 0,0.904400,0.880458,0.918250,0.822625,0.958708,0.918308,0.855508,0.976942,0.631429,0.560,0.490,0.470,0.830,0.610,0.590,0.87
Fer2013_filtered_uniform_aug_surprise100_identity05 fold 1,0.918169,0.871175,0.951533,0.864100,0.964583,0.940117,0.861192,0.974483,0.665714,0.510,0.600,0.510,0.840,0.700,0.670,0.83
Fer2013_filtered_uniform_aug_surprise100_identity05 fold 2,0.909468,0.880100,0.941883,0.847567,0.952892,0.923108,0.850592,0.970133,0.645714,0.610,0.510,0.480,0.820,0.690,0.610,0.80
Fer2013_filtered_uniform_aug_surprise100_identity05 fold 3,0.909938,0.886833,0.957050,0.821167,0.965142,0.914950,0.848417,0.976008,0.631429,0.570,0.560,0.400,0.840,0.650,0.600,0.80
Fer2013_filtered_uniform_aug_surprise100_identity05 fold 4,0.907782,0.887475,0.937867,0.830308,0.969675,0.907367,0.857875,0.963908,0.627143,0.500,0.550,0.410,0.860,0.600,0.670,0.80
Fer2013_filtered_uniform_aug_surprise100_identity05 fold 5,0.913864,0.878442,0.939583,0.868683,0.964808,0.916217,0.855383,0.973933,0.640000,0.580,0.490,0.530,0.850,0.640,0.580,0.81
Fer2013_filtered_uniform_aug_surprise100_identity05 fold 6,0.914963,0.883100,0.963817,0.855433,0.960483,0.906758,0.861767,0.973383,0.647143,0.600,0.540,0.470,0.740,0.720,0.670,0.79
Fer2013_filtered_uniform_aug_surprise100_identity05 fold 7,0.911121,0.904892,0.950367,0.805167,0.964533,0.917517,0.857467,0.977908,0.644286,0.620,0.520,0.470,0.800,0.650,0.590,0.86
Fer2013_filtered_uniform_aug_surprise100_identity05 fold 8,0.920614,0.913875,0.930550,0.861967,0.961742,0.921992,0.877317,0.976858,0.672857,0.570,0.590,0.540,0.770,0.730,0.610,0.90
Fer2013_filtered_uniform_aug_surprise100_identity05 fold 9,0.911614,0.873017,0.952383,0.841842,0.961708,0.922700,0.854000,0.975650,0.648571,0.560,0.580,0.450,0.820,0.730,0.560,0.84


## Save metrics

In [26]:
try:
    stored_df = pd.read_csv(os.path.join(metrics_directory, metrics_name),index_col=0)
except FileNotFoundError:
    stored_df = pd.DataFrame(columns = columns)
stored_df

,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean PREC,angry PREC,disgust PREC,fear PREC,happy PREC,neautral PREC,sad PREC,surprise PREC
Fer2013_filtered_uniform xval-10,0.913137,0.882769,0.947797,0.845343,0.964072,0.922568,0.854387,0.975025,0.637857,0.544,0.535,0.487,0.837,0.657,0.594,0.811
Fer2013_filtered_uniform_aug_disgust100_residual07_identity03 xval-10,0.910634,0.877640,0.948825,0.844880,0.963346,0.915373,0.848816,0.975559,0.639000,0.564,0.572,0.471,0.835,0.645,0.564,0.822
Fer2013_filtered_uniform_aug_disgust100_residual03_identity05 xval-10,0.913648,0.876134,0.948275,0.856259,0.963064,0.920421,0.857468,0.973912,0.636286,0.546,0.573,0.476,0.827,0.657,0.583,0.792
Fer2013_filtered_uniform_aug_disgust100_residual07_identity03 xval-10,0.913458,0.881824,0.944403,0.851962,0.963165,0.919623,0.855662,0.977569,0.639000,0.549,0.585,0.461,0.828,0.639,0.587,0.824
Fer2013_filtered_uniform_aug_disgust100_residual07_identity03 xval-10,0.915024,0.876282,0.948898,0.850682,0.965239,0.923594,0.863687,0.976787,0.643857,0.572,0.606,0.475,0.827,0.646,0.571,0.810
Fer2013_filtered_uniform_aug_disgust100_residual05_identity05 xval-10,0.914427,0.881619,0.949027,0.857508,0.963788,0.916292,0.859655,0.973098,0.647286,0.557,0.596,0.472,0.831,0.669,0.596,0.810
Fer2013_filtered_uniform_aug_disgust100_identity03 xval-10,0.915294,0.876576,0.952008,0.851192,0.963053,0.923192,0.862476,0.978560,0.645429,0.542,0.579,0.481,0.840,0.683,0.578,0.815
Fer2013_filtered_uniform_aug_disgust100_identity03 xval-10,0.911934,0.881547,0.950238,0.844635,0.960252,0.917508,0.853189,0.976167,0.638714,0.569,0.566,0.471,0.814,0.658,0.577,0.816
Fer2013_filtered_uniform_aug_disgust100_identity03 xval-10,0.910994,0.878423,0.947222,0.852984,0.961429,0.912841,0.850608,0.973453,0.636286,0.548,0.576,0.473,0.831,0.667,0.554,0.805
Fer2013_filtered_uniform_aug_disgust100_residual05_identity05 xval-10,0.914170,0.889486,0.947455,0.849964,0.963292,0.917965,0.856163,0.974864,0.641000,0.564,0.594,0.473,0.815,0.641,0.588,0.812


In [27]:
stored_df = pd.concat([stored_df,mean_df.set_index(pd.Index([f"{dataset_name} xval-{splits}"]))])
stored_df

,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean PREC,angry PREC,...,sad PREC,surprise PREC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Fer2013_filtered_uniform xval-10,0.913137,0.882769,0.947797,0.845343,0.964072,0.922568,0.854387,0.975025,0.637857,0.544,...,0.594,0.811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual07_identity03 xval-10,0.910634,0.877640,0.948825,0.844880,0.963346,0.915373,0.848816,0.975559,0.639000,0.564,...,0.564,0.822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual03_identity05 xval-10,0.913648,0.876134,0.948275,0.856259,0.963064,0.920421,0.857468,0.973912,0.636286,0.546,...,0.583,0.792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual07_identity03 xval-10,0.913458,0.881824,0.944403,0.851962,0.963165,0.919623,0.855662,0.977569,0.639000,0.549,...,0.587,0.824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual07_identity03 xval-10,0.915024,0.876282,0.948898,0.850682,0.965239,0.923594,0.863687,0.976787,0.643857,0.572,...,0.571,0.810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual05_identity05 xval-10,0.914427,0.881619,0.949027,0.857508,0.963788,0.916292,0.859655,0.973098,0.647286,0.557,...,0.596,0.810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_identity03 xval-10,0.915294,0.876576,0.952008,0.851192,0.963053,0.923192,0.862476,0.978560,0.645429,0.542,...,0.578,0.815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_identity03 xval-10,0.911934,0.881547,0.950238,0.844635,0.960252,0.917508,0.853189,0.976167,0.638714,0.569,...,0.577,0.816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_identity03 xval-10,0.910994,0.878423,0.947222,0.852984,0.961429,0.912841,0.850608,0.973453,0.636286,0.548,...,0.554,0.805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual05_identity05 xval-10,0.914170,0.889486,0.947455,0.849964,0.963292,0.917965,0.856163,0.974864,0.641000,0.564,...,0.588,0.812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
if not os.path.isdir(metrics_directory):
    os.makedirs(name = metrics_directory)
stored_df.to_csv(os.path.join(metrics_directory, metrics_name))

In [29]:
stored_df.index.name = "Method"
stored_df.groupby("Method").mean()

,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean PREC,angry PREC,...,sad PREC,surprise PREC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Method,,,,,,,,,,,,,,,,,,,,,
Fer2013_filtered_uniform xval-10,0.912906,0.877908,0.948334,0.848726,0.963379,0.921008,0.856463,0.974525,0.638667,0.557667,...,0.593667,0.800333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_identity0.5 xval-10,0.911370,0.877183,0.944951,0.850146,0.963337,0.917185,0.852450,0.974336,0.639238,0.554333,...,0.575333,0.798333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_identity00 xval-10,0.913287,0.879932,0.953816,0.848547,0.963040,0.914796,0.856574,0.976303,0.636381,0.554667,...,0.580667,0.815000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_identity03 xval-10,0.912741,0.878849,0.949823,0.849604,0.961578,0.917847,0.855424,0.976060,0.640143,0.553000,...,0.569667,0.812000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual03_identity05 xval-10,0.912639,0.875911,0.949864,0.848016,0.963110,0.919022,0.857366,0.975182,0.637667,0.555333,...,0.578667,0.809000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual05_identity03 xval-10,0.913626,0.884996,0.947401,0.849098,0.961179,0.920898,0.856201,0.975607,0.646810,0.570333,...,0.570333,0.804000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual05_identity05 xval-10,0.914353,0.883201,0.950140,0.852933,0.964259,0.918019,0.856888,0.975034,0.645143,0.567667,...,0.592333,0.814333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual05_identity05_alternate10 xval-10,0.911274,0.876194,0.946638,0.844182,0.961788,0.916565,0.858275,0.975274,0.636524,0.548333,...,0.587333,0.806667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fer2013_filtered_uniform_aug_disgust100_residual05_identity05_in005 xval-10,0.912101,0.879064,0.945338,0.847084,0.964124,0.917842,0.856529,0.974726,0.638810,0.561667,...,0.587667,0.809667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
winsound.Beep(400, 2000)

In [31]:
winsound.Beep(400, 2000)